In [1]:
import json
constellation_file_path = './dataset/constellation_optimal.json' 

# read the dictionary to a file in JSON format
with open(constellation_file_path, 'r') as file:
    loaded_data = json.load(file)

constellation = loaded_data["constellation"]
ground_stations = loaded_data["ground_stations"]
graph = loaded_data["graph"]

In [2]:
import networkx as nx

# Assuming 'G' is your graph, 'constellation' is your list of satellites, and 'ground_stations' is your list of ground stations.

def find_ground_station_connections(G, ground_stations, constellation, city, max_satellite_hops=None):
    """
    Finds connections from the given city to all other ground stations within a limit of satellite hops.
    Also finds all connections regardless of the number of hops.

    :param G: NetworkX graph
    :param ground_stations: List of ground stations
    :param constellation: List of satellites
    :param city: The ground station to find connections from
    :param max_satellite_hops: The maximum number of satellite hops (None for no limit on the number of hops)
    :return: Dictionary with the number of satellite hops as keys and the set of cities as values
    """
    connections = {hop: [] for hop in range(max_satellite_hops + 1)} if max_satellite_hops is not None else {}
    connections['no_limit'] = set()

    for target in ground_stations:
        if city != target:
            for path in nx.all_simple_paths(G, source=city, target=target):
                # Check if the path goes through only satellites (excluding source and target ground stations)
                if all(node in constellation for node in path[1:-1]):
                    satellite_hops = len(path) - 2  # Subtract the source and target ground stations
                    
                    # Add path to the no_limit set regardless of the number of hops
                    connections['no_limit'].add(target)

                    # If there's a hop limit, add to the appropriate list
                    if max_satellite_hops is not None and satellite_hops <= max_satellite_hops:
                        connections[satellite_hops].append(target)

    # Convert lists to sets for uniqueness
    for hop in connections:
        if hop != 'no_limit':
            connections[hop] = set(connections[hop])

    return connections



In [15]:
import networkx as nx

def find_satellite_connections(G, satellites, start_satellite, max_hops=None):
    """
    Finds connections from a given satellite to all other satellites within a limit of satellite hops.
    Also finds all connections regardless of the number of hops.

    :param G: NetworkX graph
    :param satellites: List of satellites
    :param start_satellite: The satellite from which to find connections
    :param max_hops: The maximum number of satellite hops (None for no limit on the number of hops)
    :return: Dictionary with the number of hops as keys and the set of reachable satellites as values
    """
    connections = {}
    
    if max_hops is not None:
        # Initialize dictionary for hops up to max_hops
        for hop in range(max_hops + 1):
            connections[hop] = set()
    
    # Set for collecting all reachable satellites with no limit on hops
    connections['no_limit'] = set()

    # Use BFS to find all reachable satellites within the hop limits
    for target in satellites:
        if start_satellite != target:  # Do not include the start satellite itself
            # Find all paths using breadth-first search to limit hops
            for path in nx.all_simple_paths(G, source=start_satellite, target=target, cutoff=max_hops):
                #print(path)
                num_hops = len(path) - 1  # Calculate number of hops in the path

                # Add target to the set corresponding to the number of hops
                if max_hops is None or num_hops <= max_hops:
                    connections[num_hops].add(target)
                
                # Always add to the no_limit set
                connections['no_limit'].add(target)

    return connections


In [16]:
city = 'Berlin'  # Replace with the city you're interested in
max_hops = 4 # Set to None for no limit, or an integer for a specific limit
connections = find_satellite_connections(G, constellation, city, max_hops)


In [17]:
connections 


{0: set(),
 1: {'STARLINK-1971', 'STARLINK-2024'},
 2: {'STARLINK-1144', 'STARLINK-1236', 'STARLINK-1971', 'STARLINK-2024'},
 3: {'STARLINK-1020',
  'STARLINK-1021',
  'STARLINK-1054',
  'STARLINK-1060',
  'STARLINK-1144',
  'STARLINK-1236'},
 4: {'STARLINK-1020',
  'STARLINK-1021',
  'STARLINK-1043',
  'STARLINK-1054',
  'STARLINK-1060'},
 'no_limit': {'STARLINK-1020',
  'STARLINK-1021',
  'STARLINK-1043',
  'STARLINK-1054',
  'STARLINK-1060',
  'STARLINK-1144',
  'STARLINK-1236',
  'STARLINK-1971',
  'STARLINK-2024'}}

In [6]:
import networkx as nx

G = nx.Graph()

# Add edges to the graph
for node, edges in graph.items():
    for target, weight in edges.items():
        G.add_edge(node, target, weight=weight)

# Function to find paths with up to max_satellites in between
def find_paths_with_satellites(G, ground_stations, max_satellites):
    paths = []
    for station1 in ground_stations:
        for station2 in ground_stations:
            if station1 != station2:
                all_paths = list(nx.all_simple_paths(G, station1, station2, cutoff=max_satellites+2))
                for path in all_paths:
                    if all(node not in ground_stations for node in path[1:-1]):
                        paths.append(path)
    return paths

# Check one satellite connections
one_satellite_paths = find_paths_with_satellites(G, ground_stations, 1)

# Check two satellites connections
two_satellite_paths = find_paths_with_satellites(G, ground_stations, 2)

# Compare their connection numbers
#print(f"Number of direct connections between ground stations: {len(non_conditional_connections)}")
print(f"Number of connections with exactly one satellite in between: {len(one_satellite_paths)}")
print(f"Number of connections with at most two satellites in between: {len(two_satellite_paths)}")

# Optionally, print the paths
# print("Non-Conditional Connections:", non_conditional_connections)
# print("One Satellite Paths:", one_satellite_paths)
# print("Two Satellites Paths:", two_satellite_paths)

Number of connections with exactly one satellite in between: 158
Number of connections with at most two satellites in between: 320


In [4]:
city = 'Berlin'  # Replace with the city you're interested in
max_hops = 3 # Set to None for no limit, or an integer for a specific limit
connections = find_ground_station_connections(G, ground_stations, constellation, city, max_hops)
ground_station_set = set()
# Print out the connection results
if max_hops is not None:
    for hops in range(1,max_hops + 1):
        print(f"{hops} satellite hops connections from {city}: {connections[hops]}")
        ground_station_set = ground_station_set.union(connections[hops])
print(f"No limit satellite connections from {city}: {connections['no_limit']}")


1 satellite hops connections from Berlin: {'Brussels', 'London', 'Paris', 'Amsterdam', 'Jerusalem'}
2 satellite hops connections from Berlin: {'Brussels', 'Washington D.C.', 'London', 'New York', 'Chicago', 'Ottawa', 'Paris', 'Amsterdam', 'Jerusalem', 'New Delhi'}
3 satellite hops connections from Berlin: {'Seoul', 'Los Angeles', 'Washington D.C.', 'Beijing', 'Singapore', 'New York', 'Chicago', 'Tokyo', 'Ottawa', 'New Delhi'}
No limit satellite connections from Berlin: {'Seoul', 'Brussels', 'Los Angeles', 'Washington D.C.', 'Beijing', 'London', 'Singapore', 'New York', 'Chicago', 'Tokyo', 'Ottawa', 'Paris', 'Amsterdam', 'Canberra', 'Jerusalem', 'New Delhi'}


In [5]:
city = 'Berlin'  # Replace with the city you're interested in
max_hops = 3 # Set to None for no limit, or an integer for a specific limit
connections = find_ground_station_connections(G, ground_stations, constellation, city, max_hops)
limited_ground_station_set = set()
# Print out the connection results
if max_hops is not None:
    for hops in range(1,max_hops + 1):
        
        limited_ground_station_set = limited_ground_station_set.union(connections[hops])
        print(f"{hops} satellite hops connections from {city}: {len(limited_ground_station_set)}")
print(f"No limit satellite connections from {city}: {connections['no_limit']}")


1 satellite hops connections from Berlin: 5
2 satellite hops connections from Berlin: 10
3 satellite hops connections from Berlin: 15
No limit satellite connections from Berlin: {'Seoul', 'Brussels', 'Los Angeles', 'Washington D.C.', 'Beijing', 'London', 'Singapore', 'New York', 'Chicago', 'Tokyo', 'Ottawa', 'Paris', 'Amsterdam', 'Canberra', 'Jerusalem', 'New Delhi'}


In [6]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt



In [7]:
# 加载一个包含陆地的Shapefile或GeoJSON数据
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# 生成随机坐标
num_samples = 25000
lats = np.random.uniform(-53, 53, num_samples)
lons = np.random.uniform(-180, 180, num_samples)
coords = gpd.GeoDataFrame(geometry=gpd.points_from_xy(lons, lats))

# 检查坐标是否在陆地上
coords['on_land'] = coords.within(world.unary_union)

# 过滤只在陆地上的坐标
land_coords = coords[coords['on_land']]




C:\Users\ycgao\AppData\Local\Temp\ipykernel_29536\1371964443.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [8]:
land_coords

,geometry,on_land
3,POINT (75.24041 41.43963),True
5,POINT (14.63210 17.77135),True
6,POINT (127.97863 36.07952),True
12,POINT (9.66055 22.99930),True
13,POINT (103.12057 30.59830),True
...,...,...
24941,POINT (125.49390 43.55324),True
24946,POINT (13.59923 50.10584),True
24963,POINT (92.51244 29.10357),True
24974,POINT (31.58072 51.98875),True


In [9]:
loaded_data

{'constellation': ['STARLINK-3635',
  'STARLINK-1020',
  'STARLINK-1236',
  'STARLINK-1144',
  'STARLINK-1060',
  'STARLINK-2024',
  'STARLINK-1043',
  'STARLINK-1971',
  'STARLINK-1021',
  'STARLINK-1054'],
 'ground_stations': ['Tokyo',
  'Los Angeles',
  'Canberra',
  'Jerusalem',
  'Berlin',
  'New York',
  'Amsterdam',
  'Paris',
  'London',
  'Ottawa',
  'Singapore',
  'New Delhi',
  'Beijing',
  'Washington D.C.',
  'Chicago',
  'Brussels',
  'Seoul'],
 'graph': {'Singapore': {'STARLINK-1020': 0.00014299741137104849,
   'STARLINK-1043': 0.0005259529090453201},
  'STARLINK-2024': {'STARLINK-1971': 0.0010324100781098355,
   'STARLINK-1236': 0.0005230367047495132,
   'Berlin': 0.0001629114544401806,
   'Jerusalem': 0.00017338679559758655},
  'Berlin': {'STARLINK-2024': 0.0001629114544401806,
   'STARLINK-1971': 0.00015101301303303705},
  'STARLINK-1043': {'Singapore': 0.0005259529090453201,
   'STARLINK-1060': 0.0004617030745379669,
   'STARLINK-3635': 0.0004617044502651657,
   'STA